In [ ]:
!pip install mat4py
!pip install xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 7.9 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import scipy.io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mat4py import loadmat

In [ ]:
x = loadmat('X.mat')
x

In [ ]:
data = x['X']
data


In [ ]:
list(data['rat1'].keys())

In [ ]:
cols = ['inst_freq', 'avg_freq','amplitude', 'auc','duration', 'p2p_dist', 'power', 'entropy', 'num_of_peaks', 'spect_entropy']
rats = ['rat1','rat2','rat4']
#days = {'rat1':['day20200904','day20200909','day20200911'] , 'rat2':['day20200910','day20200915','day20200917'] , 'rat4':['day20201104','day20201109','day20201111']}
hours = [2,3,4,5,1]

In [ ]:
table = pd.read_excel('table.xls')
table

,file_name,animal,study_day,goal_location,session,homecage,homecage_number,trial_name,trial,sleep,...,ripple_rate_range_75-100,ripple_rate_range_100-125,ripple_rate_range_125-150,ripple_rate_range_150-200,ripple_rate_range_200-300,ripple_av_duration,spindle_count,spindle_rate,delta_count,delta_rate
0,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep,Rat1,SD1,0,0,1,1,post_sleep1,2,1291,...,8.272657,3.067390,2.742060,2.927963,1.766073,102.359307,100,4.647560,840,39.039504
1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep,Rat1,SD1,0,0,1,1,post_sleep2,3,199,...,3.618090,1.809045,0.904523,1.507538,0.904523,109.957265,14,4.221106,117,35.276382
2,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep,Rat1,SD1,0,0,1,1,post_sleep3,4,1583,...,7.353127,3.032217,2.387871,2.539482,1.895136,103.753644,145,5.495894,858,32.520531
3,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep,Rat1,SD1,0,0,1,1,post_sleep4,5,1657,...,9.885335,4.815932,3.331322,3.983102,3.077852,107.945545,202,7.314424,594,21.508751
4,Rat_Hm_Ephys_Rat1_389236_20200904_presleep,Rat1,SD1,0,0,1,1,pre_sleep,1,1290,...,8.093023,4.232558,2.511628,2.046512,1.069767,95.346774,191,8.883721,967,44.976744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,Rat_Hm_Ephys_Rat4_389239_20201213_postsleep,Rat4,SD14,3,2,0,0,post_sleep1,2,2234,...,15.846016,10.286482,6.257833,6.230976,4.324082,106.279710,352,9.453894,861,23.124440
270,Rat_Hm_Ephys_Rat4_389239_20201213_postsleep,Rat4,SD14,3,2,0,0,post_sleep2,3,2349,...,7.586207,5.363985,3.499361,3.269476,2.247765,106.994096,352,8.991060,1009,25.772669
271,Rat_Hm_Ephys_Rat4_389239_20201213_postsleep,Rat4,SD14,3,2,0,0,post_sleep3,4,1719,...,5.933682,4.502618,2.897033,2.547993,1.884817,107.893389,243,8.481675,372,12.984293
272,Rat_Hm_Ephys_Rat4_389239_20201213_postsleep,Rat4,SD14,3,2,0,0,post_sleep4,5,2250,...,2.880000,2.400000,1.386667,1.360000,0.933333,106.533470,286,7.626667,960,25.600000


# Ripple Events

In [ ]:
dataset = pd.DataFrame()

for rat in rats:
    for d,day in enumerate(list(data[rat].keys())):
        cond = 'HC' if d == 0 else ("ENC" if d == 1 else "RET")
        hours = [2,3,4,5,1]
        if rat == 'rat4' and day =='day20201128':
          hours = [2,3,4,5]
        for h in hours:
          y = data[rat][day][f'hour{h}']
          if rat == 'rat1' and day =='day20200914' and (h == 4 or h == 5):
            y = np.array(data[rat][day][f'hour{h}']).reshape(1,10)
          x = pd.DataFrame(data = y , columns = cols)
          select_cond = (table['animal'] == f'Rat{int(rat[3])}') & (table['study_day'] == f'SD{d+1}') & (table['trial']==h)
          x['study_day'] = d+1
          x['goal_location'] = int(table.loc[select_cond,'goal_location'])
          x['session'] = int(table.loc[select_cond,'session'])
          x['homecage'] = int(table.loc[select_cond,'homecage'])
          x['homecage_number'] = int(table.loc[select_cond,'homecage_number'])
          x['trial'] = h
          x['rat'] = int(rat[3])
          df = table.copy()
          temp = df.loc[select_cond,'file_name']
          x['file_name'] = temp.to_string(index=False)
          dataset = dataset.append(x, ignore_index = True)

In [ ]:
dataset

,inst_freq,avg_freq,amplitude,auc,duration,p2p_dist,power,entropy,num_of_peaks,spect_entropy,study_day,goal_location,session,homecage,homecage_number,trial,rat,file_name
0,146.631405,137.027996,236.434594,9.681138,0.116667,421.413707,9737.337732,0.990558,16.0,4.925690,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
1,135.921003,140.991078,299.138547,7.566730,0.095000,536.385752,10697.363856,0.998001,14.0,5.215154,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
2,151.302872,148.086602,337.830980,25.835286,0.281667,558.328357,13241.920984,1.092083,44.0,5.181408,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
3,34.584912,155.957769,225.809365,7.648450,1.090000,408.791126,697.965668,0.332041,24.0,7.456011,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
4,152.099910,147.352061,257.422611,4.468201,0.060000,402.189407,8494.558142,0.979869,9.0,5.502780,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176014,151.182101,159.870533,954.477756,17.577805,0.145000,1772.152912,38294.736170,0.999905,23.0,5.458208,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
176015,166.312520,162.437959,439.296520,7.641538,0.063333,776.626837,25059.800336,1.148736,10.0,5.564759,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
176016,155.127760,178.848356,643.660492,20.895326,0.140000,1092.713483,35525.608230,0.996317,22.0,5.421289,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
176017,174.293689,173.589774,429.184266,5.975338,0.055000,642.791419,19935.470144,0.999338,9.0,5.611656,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep


In [ ]:
len(data['rat4']['day20201128'])

In [ ]:
pd.DataFrame(np.array(data['rat1']['day20200914']['hour4']).reshape(1,10))

In [ ]:
dataset.to_csv('event_chars_all.csv',index=False)
dataset.to_excel('event_chars_all.xlsx',index=False)

In [ ]:
from google.colab import files
!zip -r /content/sample_data.zip /content/sample_data
files.download('/content/sample_data.zip')

  adding: content/sample_data/ (stored 0%)
  adding: content/sample_data/event_chars_all.xlsx (deflated 5%)
  adding: content/sample_data/.ipynb_checkpoints/ (stored 0%)
  adding: content/sample_data/event_chars_all.csv (deflated 66%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
dataset.to_csv('ripple_event_chars.csv',index=False)

In [ ]:
pd.read_csv('ripple_event_chars.csv')

,inst_freq,avg_freq,amplitude,auc,duration,p2p_dist,power,entropy,num_of_peaks,spect_entropy,study_day,goal_location,session,homecage,homecage_number,trial,rat,file_name
0,146.631405,137.027996,236.434594,9.681138,0.116667,421.413707,9737.337732,0.990558,16.0,4.925690,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
1,135.921003,140.991078,299.138547,7.566730,0.095000,536.385752,10697.363856,0.998001,14.0,5.215154,1,0,0,1,1,2,1,NaN
2,151.302872,148.086602,337.830980,25.835286,0.281667,558.328357,13241.920984,1.092083,44.0,5.181408,1,0,0,1,1,2,1,NaN
3,34.584912,155.957769,225.809365,7.648450,1.090000,408.791126,697.965668,0.332041,24.0,7.456011,1,0,0,1,1,2,1,NaN
4,152.099910,147.352061,257.422611,4.468201,0.060000,402.189407,8494.558142,0.979869,9.0,5.502780,1,0,0,1,1,2,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176014,151.182101,159.870533,954.477756,17.577805,0.145000,1772.152912,38294.736170,0.999905,23.0,5.458208,14,3,2,0,0,1,4,NaN
176015,166.312520,162.437959,439.296520,7.641538,0.063333,776.626837,25059.800336,1.148736,10.0,5.564759,14,3,2,0,0,1,4,NaN
176016,155.127760,178.848356,643.660492,20.895326,0.140000,1092.713483,35525.608230,0.996317,22.0,5.421289,14,3,2,0,0,1,4,NaN
176017,174.293689,173.589774,429.184266,5.975338,0.055000,642.791419,19935.470144,0.999338,9.0,5.611656,14,3,2,0,0,1,4,NaN


In [ ]:
files.download('/content/event_chars_all.csv')

## Delta Events

In [ ]:
x_delta = loadmat('X_delta.mat')
delta = x_delta['X']

In [ ]:
list(delta['rat1'].keys())

['day20200904',
 'day20200907',
 'day20200909',
 'day20200911',
 'day20200914',
 'day20200916',
 'day20200921',
 'day20200923',
 'day20200926',
 'day20200928',
 'day20200930',
 'day20201001',
 'day20201005',
 'day20201008',
 'day20201010',
 'day20201015',
 'day20201018']

In [ ]:
delta['rat1']['day20200904']['hour2']

[[22.89984966864119,
  2.9706467203080344,
  823.2522977580527,
  140.32893720989256,
  0.31833333333333336,
  1316.3283106064434,
  239790.47388780586,
  0.9437212746306922,
  1.0,
  5.908621724255171],
 [16.754934249915866,
  3.7107118064554707,
  1388.2356068987895,
  171.02342345785434,
  0.4216666666666667,
  1538.13849549606,
  220100.5741831536,
  0.9827905978508111,
  1.0,
  6.182921131956994],
 [18.43979800256364,
  5.01603790391154,
  1064.4214314633348,
  91.9599865972243,
  0.41,
  1133.8997350179998,
  93725.68799980605,
  0.9723594643683988,
  2.0,
  6.245620956136325],
 [17.655180071352866,
  2.4216629997796524,
  672.7411309108229,
  164.17618669291357,
  0.41,
  1193.6944071279754,
  195767.39677085212,
  0.9767983353794969,
  1.0,
  5.915065922698444],
 [20.372722220772282,
  2.932183565182606,
  1291.1865701827055,
  227.93791434667605,
  0.355,
  2158.4370930290484,
  524497.290988607,
  0.9548654347204524,
  1.0,
  5.889324018585667],
 [19.852948329444754,
  7.0074

In [ ]:
delta_df = pd.DataFrame()

for rat in rats:
    for d,day in enumerate(list(delta[rat].keys())):
        cond = 'HC' if d == 0 else ("ENC" if d == 1 else "RET")
        hours = [2,3,4,5,1]
        if rat == 'rat4' and day =='day20201128':
          hours = [2,3,4,5]
        for h in hours:
          try:
            y = delta[rat][day][f'hour{h}']
            if rat == 'rat1' and day =='day20200914' and (h == 4 or h == 5):
              y = np.array(delta[rat][day][f'hour{h}']).reshape(1,10)
            x = pd.DataFrame(data = y , columns = cols)
            select_cond = (table['animal'] == f'Rat{int(rat[3])}') & (table['study_day'] == f'SD{d+1}') & (table['trial']==h)
            x['study_day'] = d+1
            x['goal_location'] = int(table.loc[select_cond,'goal_location'])
            x['session'] = int(table.loc[select_cond,'session'])
            x['homecage'] = int(table.loc[select_cond,'homecage'])
            x['homecage_number'] = int(table.loc[select_cond,'homecage_number'])
          except:
            print(f'Rat: {rat}, day: {day}, hour: {h}')
          x['trial'] = h
          x['rat'] = int(rat[3])
          df = table.copy()
          temp = df.loc[select_cond,'file_name']
          x['file_name'] = temp.to_string(index=False)
          delta_df = delta_df.append(x, ignore_index = True)

### *Rat1 has only one delta event in the hour 4 & 5 of day20200914*
### *Rat4 doesn't have hour 1 for day20201128*
###Rat: rat1, day: day20200914, hour: 4<br>Rat: rat1, day: day20200914, hour: 5<br>Rat: rat4, day: day20201128, hour: 1

In [ ]:
delta_df

,inst_freq,avg_freq,amplitude,auc,duration,p2p_dist,power,entropy,num_of_peaks,spect_entropy,study_day,goal_location,session,homecage,homecage_number,trial,rat,file_name
0,22.899850,2.970647,823.252298,140.328937,0.318333,1316.328311,2.397905e+05,0.943721,1.0,5.908622,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
1,16.754934,3.710712,1388.235607,171.023423,0.421667,1538.138495,2.201006e+05,0.982791,1.0,6.182921,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
2,18.439798,5.016038,1064.421431,91.959987,0.410000,1133.899735,9.372569e+04,0.972359,2.0,6.245621,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
3,17.655180,2.421663,672.741131,164.176187,0.410000,1193.694407,1.957674e+05,0.976798,1.0,5.915066,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
4,20.372722,2.932184,1291.186570,227.937914,0.355000,2158.437093,5.244973e+05,0.954865,1.0,5.889324,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152449,16.925441,2.478930,4351.860283,661.410993,0.420000,5458.091186,3.567456e+06,0.998864,1.0,5.950510,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
152450,17.093416,2.521865,3880.199730,776.127178,0.425000,6273.986872,4.260201e+06,0.999900,1.0,6.422904,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
152451,16.643853,2.347056,3977.496987,969.375102,0.440000,6784.388712,5.946805e+06,1.029799,1.0,6.843825,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
152452,15.908318,2.416280,3253.961895,680.993414,0.445000,5268.970143,3.024068e+06,0.981209,1.0,6.828092,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep


In [ ]:
delta['rat4']['day20201128'].keys()

dict_keys(['hour2', 'hour3', 'hour4', 'hour5'])

In [ ]:
delta_df.to_csv('delta_event_chars.csv',index=False)

In [ ]:
delta_df.to_excel('delta_event_chars.xlsx',index=False)

In [ ]:
from google.colab import files

In [ ]:
# files.download('delta_event_chars.csv')
files.download('delta_event_chars.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pd.read_csv('delta_event_chars.csv')

,inst_freq,avg_freq,amplitude,auc,duration,p2p_dist,power,entropy,num_of_peaks,spect_entropy,study_day,goal_location,session,homecage,homecage_number,trial,rat,file_name
0,22.899850,2.970647,823.252298,140.328937,0.318333,1316.328311,2.397905e+05,0.943721,1.0,5.908622,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
1,16.754934,3.710712,1388.235607,171.023423,0.421667,1538.138495,2.201006e+05,0.982791,1.0,6.182921,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
2,18.439798,5.016038,1064.421431,91.959987,0.410000,1133.899735,9.372569e+04,0.972359,2.0,6.245621,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
3,17.655180,2.421663,672.741131,164.176187,0.410000,1193.694407,1.957674e+05,0.976798,1.0,5.915066,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
4,20.372722,2.932184,1291.186570,227.937914,0.355000,2158.437093,5.244973e+05,0.954865,1.0,5.889324,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152449,16.925441,2.478930,4351.860283,661.410993,0.420000,5458.091186,3.567456e+06,0.998864,1.0,5.950510,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
152450,17.093416,2.521865,3880.199730,776.127178,0.425000,6273.986872,4.260201e+06,0.999900,1.0,6.422904,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
152451,16.643853,2.347056,3977.496987,969.375102,0.440000,6784.388712,5.946805e+06,1.029799,1.0,6.843825,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
152452,15.908318,2.416280,3253.961895,680.993414,0.445000,5268.970143,3.024068e+06,0.981209,1.0,6.828092,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep


## Spindle Events

In [ ]:
x_spd = loadmat('X_spindle.mat')
spindle = x_spd['X']

In [ ]:
spindle_df = pd.DataFrame()

for rat in rats:
    for d,day in enumerate(list(spindle[rat].keys())):
        cond = 'HC' if d == 0 else ("ENC" if d == 1 else "RET")
        hours = [2,3,4,5,1]
        if rat == 'rat4' and day =='day20201128':
          hours = [2,3,4,5]
        for h in hours:
          try:
            y = spindle[rat][day][f'hour{h}']
            if rat == 'rat1' and day =='day20200914' and (h == 4 or h == 5):
              y = np.array(spindle[rat][day][f'hour{h}']).reshape(1,10)
            x = pd.DataFrame(data = y , columns = cols)
            select_cond = (table['animal'] == f'Rat{int(rat[3])}') & (table['study_day'] == f'SD{d+1}') & (table['trial']==h)
            x['study_day'] = d+1
            x['goal_location'] = int(table.loc[select_cond,'goal_location'])
            x['session'] = int(table.loc[select_cond,'session'])
            x['homecage'] = int(table.loc[select_cond,'homecage'])
            x['homecage_number'] = int(table.loc[select_cond,'homecage_number'])
          except:
            print(f'Rat: {rat}, day: {day}, hour: {h}')
          x['trial'] = h
          x['rat'] = int(rat[3])
          df = table.copy()
          temp = df.loc[select_cond,'file_name']
          x['file_name'] = temp.to_string(index=False)
          spindle_df = spindle_df.append(x, ignore_index = True)

### *Rat1 has only one spindle event in the hour 4 & 5 of day20200914*
### *Rat4 doesn't have hour 1 for day20201128*
###Rat: rat1, day: day20200914, hour: 4<br>Rat: rat1, day: day20200914, hour: 5<br>Rat: rat4, day: day20201128, hour: 1

In [ ]:
spindle_df

,inst_freq,avg_freq,amplitude,auc,duration,p2p_dist,power,entropy,num_of_peaks,spect_entropy,study_day,goal_location,session,homecage,homecage_number,trial,rat,file_name
0,13.281371,11.798819,1145.761757,267.551615,0.931667,2093.093741,137308.465114,1.016524,11.0,6.910597,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
1,12.407740,12.901360,814.799201,393.546528,1.565000,1571.289990,94174.623861,1.020456,19.0,8.260873,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
2,13.635849,11.693966,1039.843251,305.662445,0.885000,1957.650412,181392.913718,0.998400,10.0,7.862812,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
3,11.664172,12.520610,988.003322,404.384867,1.288333,1891.327205,143944.666370,1.012746,15.0,8.004483,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
4,11.724686,11.923202,1007.573860,893.966812,3.008333,1953.892265,134644.540240,1.024870,38.0,9.153444,1,0,0,1,1,2,1,Rat_Hm_Ephys_Rat1_389236_20200904_postsleep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33714,17.333024,11.844358,575.628318,101.107836,0.528333,1105.133150,56594.612070,0.998384,5.0,7.264117,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
33715,15.998027,11.748941,627.328476,102.525354,0.558333,1174.437917,54516.562276,0.999994,5.0,6.865859,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
33716,15.150626,11.713399,513.422333,142.710530,0.705000,971.702637,55926.921450,0.999319,9.0,6.949230,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep
33717,11.998335,11.877158,648.819157,223.233789,1.416667,1198.058937,40301.930585,1.011847,17.0,8.267756,14,3,2,0,0,1,4,Rat_Hm_Ephys_Rat4_389239_20201213_presleep


In [ ]:
spindle_df.to_csv('spindle_event_chars.csv',index=False)

In [ ]:
spindle_df.to_excel('spindle_event_chars.xlsx',index=False)